1. Establish a connection between Python and the Sakila database.

In [5]:
import mysql.connector

# run in anaconda prompt : pip install mysql-connector-python

In [65]:
from getpass import getpass

password = getpass()
cnx = mysql.connector.connect(user="root",
                              password=password,
                              host="localhost",
                              database="sakila")

cnx.is_connected()

········


True

2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

In [66]:
cursor = cnx.cursor()

In [67]:
import pandas as pd

def rentals_month(db,month,year):
    quary = (f"""select *
                from {db}
                where month(rental_date) = {month} and year(rental_date) = {year};""")
    cursor.execute(quary)
    results = cursor.fetchall()
    columns = [header[0] for header in cursor.description]
    rental_df = pd.DataFrame(results, columns=columns)
    rental_df = rental_df.drop(["month","weekday"], axis=1)
    return rental_df

In [71]:
rentals_month("sakila.rental",5,2005)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2023-11-01 01:41:51
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2023-11-01 01:41:51
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2023-11-01 01:41:51
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2023-11-01 01:41:51
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2023-11-01 01:41:51
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2023-11-01 01:41:51
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2023-11-01 01:41:51
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2023-11-01 01:41:51
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2023-11-01 01:41:51


3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

In [79]:
def rental_count_month(df,month,year):
    df['rental_date'] = pd.to_datetime(df['rental_date'])
    filtered_data = df[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)]
    filtered_data = filtered_data.groupby('customer_id')['rental_id'].agg('count').reset_index()
    filtered_data = filtered_data.rename(columns = {'rental_id':'rental_count'})
    return filtered_data

In [80]:
rental_df = rentals_month("sakila.rental",5,2005)
rental_count_month(rental_df,5,2005)

,customer_id,rental_count
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [82]:
def compare_rentals(df1,df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer')

    merged_df['rental_count_x'].fillna(0, inplace=True)
    merged_df['rental_count_y'].fillna(0, inplace=True)

    merged_df['difference'] = merged_df['rental_count_x'] - merged_df['rental_count_y']

    return merged_df

In [83]:
rental_df1 = rentals_month("sakila.rental",5,2005)
rental_df2 = rentals_month("sakila.rental",2,2006)
df1 = rental_count_month(rental_df1,5,2005)
df2 = rental_count_month(rental_df2,2,2006)
compare_rentals(df1,df2)

,customer_id,rental_count_x,rental_count_y,difference
0,1,2.0,0.0,2.0
1,2,1.0,0.0,1.0
2,3,2.0,0.0,2.0
3,5,3.0,1.0,2.0
4,6,3.0,0.0,3.0
...,...,...,...,...
534,527,0.0,1.0,-1.0
535,568,0.0,1.0,-1.0
536,579,0.0,1.0,-1.0
537,585,0.0,1.0,-1.0


In [84]:
cursor.close()
cnx.close()